In [1]:
import json
import numpy as np
import imageio
import os
import tqdm
from glob import glob
import shutil
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[12, 12]
from matplotlib.patches import Polygon
from PIL import Image, ImageDraw, ImageOps
import cv2 as cv2
import json
import requests
from datetime import datetime

In [2]:
def find_poly_in_instance(img_filename, label_dict):
    shape_attributes = label_dict.get(img_filename)
    polygons = []
    for sh_attr in shape_attributes:
        xy = list(zip(sh_attr.get('shape_attributes')['all_points_x'], sh_attr.get('shape_attributes')['all_points_y']))
        polygons.append(xy)

    return polygons

In [3]:
url = " "
response = requests.get(url)
data = response.json()

In [4]:
len(data)

32

In [5]:
datetime.now()
now = datetime.now()
dt_string = now.strftime("%Y%m%d-%H%M%S")

folder_name = dt_string
export_path = os.path.join('/media/scanvan/web/public/reporting', dt_string)
os.makedirs(export_path, exist_ok=True)

In [6]:
root_path = '/media/scanvan/web/public/test/'
SRCS = []
MASKS = []
for i in tqdm.tqdm_notebook(range(len(data))):
    
    im_path = root_path + data[i]['filepath']
    SRCS.append(im_path)
    im = Image.open(im_path)
    width, height  = im.size
    
    poly = []
    for order in data[i]['pitchYawOrders']:
        y = height * order['pitch'] / np.pi
        x = width * order['yaw'] / ( 2 * np.pi )
        poly.append( ( x , y ) )
        
    mask = Image.new('L', (width, height), 0)
    ImageDraw.Draw(mask).polygon(poly, outline=1, fill=1)
    mask_inv = cv2.bitwise_not(np.array(mask) * 255)
    
    mask_path = os.path.join('/media/scanvan/web/public/reporting', dt_string, data[i]['viewpointId'] + "-mask.jpg")
    MASKS.append(mask_path)
    imageio.imwrite(mask_path, mask_inv)
    
    # images are re-written at the same path
    !/home/descombe/Sources/image-suite/bin/image-missing-pixel -s $im_path -m $mask_path -e $im_path -k 16
        

<ipython-input-6-2e5cefc5926c>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm.tqdm_notebook(range(len(data))):


In [8]:
# can be useful to check new annotations
for mask in MASKS:
    !cp $mask .

In [ ]:
# useful to check annotations
extent = 0, width, 0, height
fig = plt.figure(frameon=False, figsize = (23,23))

im1 = plt.imshow(im,
                 extent=extent)

im2 = plt.imshow(mask,
                 extent=extent, alpha=0.5)

plt.show()